In [10]:
##Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 500)
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker


# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.compose import make_column_transformer


import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy import stats
from scipy.stats import norm, kurtosis, skew

# Import specific libraries
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('expand_frame_repr', False)



In [11]:
#Loading updated dataset

#all_data = pd.read_csv("data_6.csv", index_col = 0) #Working Ok
#all_data = all_data.fillna("NA") #por alguna razón cuando leo el dataset desde house me reemplaza los NA con nan y me caga

In [12]:
%store -r all_data #load the dataset from house price 4 notebook!

In [13]:
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalPoints,TotalExtraPoints
0,60,RL,65.0000,8450,1,NA,4,Lvl,Inside,3,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0000,4,3,PConc,4.0000,3.0000,1.0000,6.0000,706.0000,1.0000,0.0000,150.0000,856.0000,GasA,5,1,SBrkr,856,854,0,1710,1.0000,0.0000,2,1,3,1,4,8,7,0,0,Attchd,2003.0000,2,2.0000,548.0000,3,3,3,0,61,0,0,0,0,0,0,NA,0,2,2008,WD,Normal,91.0000,9
1,20,RL,80.0000,9600,1,NA,4,Lvl,FR2,3,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0000,3,3,CBlock,4.0000,3.0000,4.0000,5.0000,978.0000,1.0000,0.0000,284.0000,1262.0000,GasA,5,1,SBrkr,1262,0,0,1262,0.0000,1.0000,2,0,3,1,3,6,7,1,3,Attchd,1976.0000,2,2.0000,460.0000,3,3,3,298,0,0,0,0,0,0,0,NA,0,5,2007,WD,Normal,84.0000,11
2,60,RL,68.0000,11250,1,NA,3,Lvl,Inside,3,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0000,4,3,PConc,4.0000,3.0000,2.0000,6.0000,486.0000,1.0000,0.0000,434.0000,920.0000,GasA,5,1,SBrkr,920,866,0,1786,1.0000,0.0000,2,1,3,1,4,6,7,1,3,Attchd,2001.0000,2,2.0000,608.0000,3,3,3,0,42,0,0,0,0,0,0,NA,0,9,2008,WD,Normal,95.0000,12
3,70,RL,60.0000,9550,1,NA,3,Lvl,Corner,3,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0000,3,3,BrkTil,3.0000,4.0000,1.0000,5.0000,216.0000,1.0000,0.0000,540.0000,756.0000,GasA,4,1,SBrkr,961,756,0,1717,1.0000,0.0000,1,0,3,1,4,7,7,1,4,Detchd,1998.0000,1,3.0000,642.0000,3,3,3,0,35,272,0,0,0,0,0,NA,0,2,2006,WD,Abnorml,92.0000,12
4,60,RL,84.0000,14260,1,NA,3,Lvl,FR2,3,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0000,4,3,PConc,4.0000,3.0000,3.0000,6.0000,655.0000,1.0000,0.0000,490.0000,1145.0000,GasA,5,1,SBrkr,1145,1053,0,2198,1.0000,0.0000,2,1,4,1,4,9,7,1,3,Attchd,2000.0000,2,3.0000,836.0000,3,3,3,192,84,0,0,0,0,0,0,NA,0,12,2008,WD,Normal,111.0000,12


In [14]:
##Se crea un feature con todos los ordinales, los cuales son relativos a la calidad de la infraestructura
all_data["TotalPoints"] = (all_data.ExterQual + all_data.FireplaceQu + all_data.GarageQual + all_data.KitchenQual +
                            all_data.BsmtQual + all_data.BsmtExposure + all_data.BsmtFinType1 + all_data.PoolQC + 
                            all_data.ExterCond + all_data.BsmtCond + all_data.GarageCond + all_data.OverallCond +
                            all_data.BsmtFinType2 + all_data.HeatingQC) + all_data.OverallQual**2

##Se crea un feature que agrega los puntajes de todos los extras dentro la infarestructura
all_data["TotalExtraPoints"] = all_data.HeatingQC + all_data.PoolQC + all_data.FireplaceQu  + all_data.KitchenQual



In [21]:
##Plotting for correlation visualization with newly created features
def plot_vars(cols, t, target):
    sns.reset_defaults()
    sns.set(style="ticks", color_codes=True)
    fig = plt.figure(figsize = (10,20))
    i = 1
    for col in cols:
        fig_i= fig.add_subplot(420 + i)
        sns.regplot( x= col, y = target, data = t )
        plt.title("Correlation to target:{:1.4f}".format(t[col].corr(t[target])))
        i +=1

In [22]:
%store -r y_train #loading target

no stored variable y_train,
no stored variable %store - r train


In [26]:
%store -r train #loading train

In [36]:
len(y_train)

1460

In [41]:
all_data.iloc[:len(y_train), :]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalPoints,TotalExtraPoints
0,60,RL,65.0000,8450,1,NA,4,Lvl,Inside,3,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0000,4,3,PConc,4.0000,3.0000,1.0000,6.0000,706.0000,1.0000,0.0000,150.0000,856.0000,GasA,5,1,SBrkr,856,854,0,1710,1.0000,0.0000,2,1,3,1,4,8,7,0,0,Attchd,2003.0000,2,2.0000,548.0000,3,3,3,0,61,0,0,0,0,0,0,NA,0,2,2008,WD,Normal,91.0000,9
1,20,RL,80.0000,9600,1,NA,4,Lvl,FR2,3,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0000,3,3,CBlock,4.0000,3.0000,4.0000,5.0000,978.0000,1.0000,0.0000,284.0000,1262.0000,GasA,5,1,SBrkr,1262,0,0,1262,0.0000,1.0000,2,0,3,1,3,6,7,1,3,Attchd,1976.0000,2,2.0000,460.0000,3,3,3,298,0,0,0,0,0,0,0,NA,0,5,2007,WD,Normal,84.0000,11
2,60,RL,68.0000,11250,1,NA,3,Lvl,Inside,3,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0000,4,3,PConc,4.0000,3.0000,2.0000,6.0000,486.0000,1.0000,0.0000,434.0000,920.0000,GasA,5,1,SBrkr,920,866,0,1786,1.0000,0.0000,2,1,3,1,4,6,7,1,3,Attchd,2001.0000,2,2.0000,608.0000,3,3,3,0,42,0,0,0,0,0,0,NA,0,9,2008,WD,Normal,95.0000,12
3,70,RL,60.0000,9550,1,NA,3,Lvl,Corner,3,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0000,3,3,BrkTil,3.0000,4.0000,1.0000,5.0000,216.0000,1.0000,0.0000,540.0000,756.0000,GasA,4,1,SBrkr,961,756,0,1717,1.0000,0.0000,1,0,3,1,4,7,7,1,4,Detchd,1998.0000,1,3.0000,642.0000,3,3,3,0,35,272,0,0,0,0,0,NA,0,2,2006,WD,Abnorml,92.0000,12
4,60,RL,84.0000,14260,1,NA,3,Lvl,FR2,3,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0000,4,3,PConc,4.0000,3.0000,3.0000,6.0000,655.0000,1.0000,0.0000,490.0000,1145.0000,GasA,5,1,SBrkr,1145,1053,0,2198,1.0000,0.0000,2,1,4,1,4,9,7,1,3,Attchd,2000.0000,2,3.0000,836.0000,3,3,3,192,84,0,0,0,0,0,0,NA,0,12,2008,WD,Normal,111.0000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,20,RL,70.0000,8400,1,NA,4,Lvl,Corner,3,NAmes,Norm,Norm,1Fam,1Story,4,5,1970,1970,Gable,CompShg,Plywood,Plywood,None,0.0000,3,3,CBlock,3.0000,3.0000,1.0000,5.0000,804.0000,3.0000,78.0000,0.0000,882.0000,GasA,3,1,SBrkr,882,0,0,882,1.0000,0.0000,1,0,2,1,3,4,7,0,0,Attchd,1970.0000,3,2.0000,525.0000,3,3,3,240,0,0,0,0,0,0,3,NA,0,4,2010,WD,Normal,54.0000,6
10,120,RH,26.0000,5858,1,NA,3,Lvl,FR2,3,NAmes,Norm,Norm,TwnhsE,1Story,7,5,1999,1999,Gable,CompShg,MetalSd,MetalSd,None,0.0000,4,3,PConc,4.0000,3.0000,1.0000,6.0000,1051.0000,4.0000,354.0000,354.0000,1405.0000,GasA,5,1,SBrkr,1337,0,0,1337,1.0000,0.0000,2,0,2,1,4,5,7,1,2,Attchd,1999.0000,3,2.0000,511.0000,3,3,3,203,68,0,0,0,0,0,0,NA,0,6,2010,WD,Normal,96.0000,11
11,160,RM,21.0000,1680,1,NA,4,Lvl,Inside,3,BrDale,Norm,Norm,Twnhs,2Story,6,5,1971,1971,Gable,CompShg,HdBoard,HdBoard,BrkFace,504.0000,3,3,CBlock,3.0000,3.0000,1.0000,3.0000,156.0000,1.0000,0.0000,327.0000,483.0000,GasA,3,1,SBrkr,483,504,0,987,0.0000,0.0000,1,1,2,1,3,5,7,0,0,Det

In [43]:
t = pd.concat((all_data.iloc[:len(y_train), :], y_train), axis = 1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [30]:
plot_vars(["HeatingQC", "PoolQC", "FireplaceQu", "KitchenQual"], t = all_data[:y_train.shape[0]], target = "SalePrice")

KeyError: 'SalePrice'